In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import seaborn as sns
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

In [ ]:
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
week_nums = [210605, 210612 ,210619, 210626, 210703,210710,210717,210724,210731,210807,210814,210821,210828]
turnstiles_data = get_data(week_nums)

In [ ]:
turnstiles_data["DATETIME"] = pd.to_datetime(turnstiles_data.DATE + " " + turnstiles_data.TIME, 
                                            format="%m/%d/%Y %H:%M:%S") 

In [ ]:
turnstiles_data['DATE'] = pd.to_datetime(turnstiles_data['DATE'])
turnstiles_data['DayOfWeek'] = turnstiles_data['DATE'].dt.day_name()

In [ ]:
turnstiles_data.columns = [column.strip() for column in turnstiles_data.columns]
turnstiles_data.columns

In [ ]:
turnstiles_data.sample(20)

In [ ]:
engine = create_engine("sqlite:///my_database.db")
engine.table_names()
turnstiles_data.to_sql('mtaColumns', engine, if_exists='replace', index=False)
turnstiles_data = pd.read_sql('SELECT * FROM mtaColumns;', engine)

In [ ]:
turnstiles_data.sample(10)

# EDA

In [ ]:
turnstiles_data.info()

In [ ]:
turnstiles_data.info()

In [ ]:
turnstiles_data.isnull().sum()

In [ ]:
turnstiles_data.describe()

In [ ]:
(turnstiles_data
.groupby(["C/A", "UNIT", "SCP", "STATION", "DATETIME"])
.ENTRIES.count()
.reset_index()
.sort_values("ENTRIES", ascending=False)).head(20)

In [ ]:
import datetime as dt
my_datetime = dt.datetime(2021, 7, 22)
mask = ((turnstiles_data["C/A"] == "R514") & 
(turnstiles_data["UNIT"] == "R094") & 
(turnstiles_data["SCP"] == "00-05-00") & 
(turnstiles_data["STATION"] == "ASTORIA BLVD") &
(turnstiles_data["DATETIME"].dt.date == my_datetime.date()))

turnstiles_data[mask]

In [ ]:
turnstiles_data.DESC.value_counts()

In [ ]:
turnstiles_data.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATETIME"], 
                          inplace=True, ascending=False)
turnstiles_data.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATETIME"], inplace=True)

In [ ]:
(turnstiles_data
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATETIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(50)

In [ ]:
turnstiles_data.shape 

In [ ]:
turnstiles_data.sample(20)

In [ ]:
turnstiles_data.reset_index(inplace=True)

# Analysis 

In [ ]:
turnstiles_data['TRAFFIC']= turnstiles_data['ENTRIES']+ turnstiles_data['EXITS']

In [ ]:
turnstiles_data[["PREV_DATE", "PREV_TRAFFIC"]] = (turnstiles_data
                                     .groupby(["C/A", "UNIT", "SCP", "STATION","DATE"])["DATETIME", "TRAFFIC"]
                                    .apply(lambda grp: grp.shift(1)))

In [ ]:
turnstiles_data.dropna(subset=["PREV_DATE"], axis=0, inplace=True)
(turnstiles_data[turnstiles_data["TRAFFIC"] < turnstiles_data["PREV_TRAFFIC"]]
    .groupby(["C/A", "UNIT", "SCP", "STATION"])
    .size())

In [ ]:
def get_traffic_counts(row, max_counter):
    counter = row["TRAFFIC"] - row["PREV_TRAFFIC"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0?
        print(row["TRAFFIC"], row["PREV_TRAFFIC"])
        counter = min(row["TRAFFIC"], row["PREV_TRAFFIC"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter
# If counter is > 1Million, then the counter might have been reset.
# Just set it to zero as different counters have different cycle limits
# It'd probably be a good idea to use a number even significantly smaller than 1 million as the limit!
turnstiles_data["DAILY_TRAFFIC"] = turnstiles_data.apply(get_traffic_counts, axis=1, max_counter=100000)
#turnstiles_daily["DAILY_ENTRIES"] = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000)

In [ ]:
sd = turnstiles_data.groupby(["STATION", "DayOfWeek"])[['DAILY_TRAFFIC']].sum().reset_index()
f = sd.sort_values('DAILY_TRAFFIC')

In [ ]:
mask1=sd[(sd['STATION']=='34 ST-PENN STA') | (sd['STATION']=='GRD CNTRL-42 ST') | (sd['STATION']=='34 ST-HERALD SQ') | (sd['STATION']=='86 ST') | (sd['STATION']=='14 ST-UNION SQ')]  

mask22 = mask1.groupby(['STATION', 'DayOfWeek']).agg({'DAILY_TRAFFIC': sum})

mask22

In [ ]:
station_totals = sd.groupby('STATION').sum()\
    .sort_values('DAILY_TRAFFIC', ascending=False)\
    .reset_index()

In [ ]:
station_totals

In [ ]:
top5 = mask22.groupby(['STATION']).agg({'DAILY_TRAFFIC': sum}).sort_values(by = 'DAILY_TRAFFIC', ascending = False).head(5)

# Visualization

In [ ]:
top_5 = station_totals.groupby(['STATION']).agg({'DAILY_TRAFFIC': sum}).sort_values(by = 'DAILY_TRAFFIC', ascending = False).head(5)
fig, ax = plt.subplots(figsize=(10, 5))
top_5.sort_values(by = 'DAILY_TRAFFIC',ascending=True).plot(kind='bar', color ='orange', ax=ax)
ax.set(title='Top 5 Stations by Turnstiles Traffic (June to August 2021)', xlabel='Stations', ylabel=' Traffic')
plt.xticks(rotation=45)
ax.legend().set_visible(False)

In [ ]:
df = mask22.groupby(['DayOfWeek']).agg({'DAILY_TRAFFIC': sum}).sort_values(by = 'DAILY_TRAFFIC', ascending = False)
fig, ax = plt.subplots(figsize=(10, 5))
df.sort_values(by = 'DAILY_TRAFFIC',ascending=True).plot(kind='bar', ax=ax)
ax.set(title='Top 5 Stations by Turnstiles Traffic (June to August 2021)', xlabel='Stations', ylabel=' Traffic')
plt.xticks(rotation=45) 
ax.legend().set_visible(False)

In [ ]:
ax = sns.barplot(x="DAILY_TRAFFIC", y="DayOfWeek",data=f, palette="Blues_d", ci = None)
ax.set_yticklabels(ax.get_yticklabels(),rotation = 15);
ax.set_xlabel("Daily Traffic", fontsize = 15)
ax.set_ylabel("Days of The week", fontsize = 15);


In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
c= sns.lineplot(data=mask22, x="DayOfWeek", y="DAILY_TRAFFIC", hue="STATION")
c.set_ylabel("Daily Traffic", fontsize = 15)
c.set_xlabel("Days of The week", fontsize = 15);

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))
colormap = sns.color_palette("Blues",12)
v=sns.heatmap(top5, annot = True, cmap=colormap)
v.set_yticklabels(ax.get_yticklabels(),rotation = 15);
v.set_ylabel("Stations", fontsize = 15)
plt.xlabel('Daily Traffic')
plt.show()

In [ ]:
i = sns.heatmap(top5)
i.set_yticklabels(ax.get_yticklabels(),rotation = 15);
i.set_ylabel("Stations", fontsize = 15)
i.set_xlabel("Daily Traffic", fontsize = 15);

In [ ]:
plt.figure(figsize=(20,5))
v= sd.set_index('DayOfWeek')['DAILY_TRAFFIC'].plot(color = 'steelblue')
plt.title('Daily Total Traffic for June to August 2021') 
v.set_ylabel("Daily Traffic", fontsize = 15)
v.set_xlabel("days", fontsize = 15);
plt.show()